## SET-NET Birth Defects NLP Python Code: Jupyter Notebook

Authors: Suzy Newton, Nicki Roth, Amy Board, Samantha Distler \
Last Updated: 08/16/2023 \
Purpose: To categorize free text fields for birth defects into overarching categories for analysis \
\
This notebook will guide you through the steps for conducting text cleaning and categorization of the bg_icd and bg_icd_sp fields.

## Libraries and setup

Before running the cell below, make sure that you have installed the packages using the Anaconda command prompt. If you do not have Anaconda installed, you may download it here: https://www.anaconda.com/products/distribution. \
\
When running the cell below, you may get the following warning message: 'UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning.' You can ignore this message--it's in reference to the fuzzy matching process function we'll use later on, and it's just warning you that the fuzzy match might take a little while to run.

In [1]:
import pandas as pd
import numpy as np
import os
import re
import sys
import string
import seaborn as sns
import matplotlib.pyplot as plt


# text processing and NLP
import nltk
import sklearn
from nltk.tokenize import RegexpTokenizer,sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

# fuzzy matching
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# So all output comes through from Ipython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Set the max number of rows to print out so the notebook is easier to read and navigate
# You can increase this beyond 20 if you want
pd.options.display.max_rows = 20

#Connecting to the SQL server
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

## Check your working directory

Your working directory should point to wherever you cloned this GitHub repo. You can check your working directory like so:

In [2]:
# Working Directory
print(os.getcwd())

C:\Users\ogc4\Desktop


In [ ]:
# Set Working Directory (if needed)
## os.chdir("[GITHUB-REPO-LOCATION-HERE]") 

In [3]:
# Confirm it changed the working Directory
print("My working directory:\n" + os.getcwd())

My working directory:
C:\Users\ogc4\Desktop


## Import the dummy data file

The NLP folder in this GitHub Repo contains a file of dummy data that can be used for learning purposes. If you are using this code with other data, you may need to clean and transform your dataset so it matches this format.

In [4]:
data = pd.read_excel('synthetic_data_062023.xlsx')

data=data[['ID', 'bg_icd', 'bg_icd_sp']] #extra step to remove temporary notes columns

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         1000 non-null   int64 
 1   bg_icd     876 non-null    object
 2   bg_icd_sp  875 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


,ID,bg_icd,bg_icd_sp
0,1,Q826,Congenital sacral dimple
1,2,M95.2,"Head, molding noted."
2,3,751610,Mesothelial liver cyst
3,4,753410,Right duplex kidney
4,5,Q540,"Hypospadias, balanic"


## Import and clean the ICD code reference list

Now we need to import and clean the ICD code reference list so that we can match ICD codes and text data from the POB file and categorize them accordingly. Be sure to update the file directory and/or filename as needed in the step below to make sure that the most current, up-to-date reference list is being pulled in.

In [23]:
q = pd.read_excel('SETNET_birthdefects_codes_inclusion.xlsx')

q.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ICD         924 non-null    object 
 1   MACDP_low   137 non-null    float64
 2   MACDP_high  137 non-null    float64
 3   Text        924 non-null    object 
 4   Category    924 non-null    object 
dtypes: float64(2), object(3)
memory usage: 36.2+ KB


In [6]:
# Display the first 5 observations in the reference list
# Check against the excel spreadsheet to ensure that everything looks correct

q.head()

,ICD,MACDP_low,MACDP_high,Text,Category
0,Q00,NaN,NaN,Anencephaly and similar malformations,Congenital malformations of the nervous system
1,Q00.0,740000.0,740109.0,Anencephaly,Congenital malformations of the nervous system
2,Q00.1,NaN,NaN,Craniorachischisis,Congenital malformations of the nervous system
3,Q00.2,NaN,NaN,Iniencephaly,Congenital malformations of the nervous system
4,Q01,742000.0,742099.0,Encephalocele,Congenital malformations of the nervous system


In [7]:
# Count the number of categories in the reference list
# The resulting number should be 13 (1 more category will be added in later steps)

q['Category'].nunique()

12

In [24]:
# remove periods and any other characters that might have been mistakenly added to the ICD code field
remove = ['\\.', ' ', ',', ';', ':', '_', '\\?', '!', '\\(', '\\)']

for i in remove:
    q['ICD'] = q['ICD'].str.replace(i,'',regex=True)


q


,ICD,MACDP_low,MACDP_high,Text,Category
0,Q00,NaN,NaN,Anencephaly and similar malformations,Congenital malformations of the nervous system
1,Q000,740000.0,740109.0,Anencephaly,Congenital malformations of the nervous system
2,Q001,NaN,NaN,Craniorachischisis,Congenital malformations of the nervous system
3,Q002,NaN,NaN,Iniencephaly,Congenital malformations of the nervous system
4,Q01,742000.0,742099.0,Encephalocele,Congenital malformations of the nervous system
...,...,...,...,...,...
919,Q990,NaN,NaN,"Chimera 46, XX/46, XY","Chromosomal abnormalities, not elsewhere class..."
920,Q991,NaN,NaN,"46, XX true hermaphrodite","Chromosomal abnormalities, not elsewhere class..."
921,Q992,NaN,NaN,Fragile X chromosome,"Chromosomal abnormalities, not elsewhere class..."
922,Q998,NaN,NaN,Other specified chromosome abnormalities,"Chromosomal abnormalities, not elsewhere class..."


## Explore and modify the POB data

The steps below provide insight into the basic data structure, data types, and data composition. They also clean the bg_icd and bg_icd_sp codes in preparation for matching to the reference list.

In [25]:
# Identify the number of null values in each column
# If there are null (NaN) values in the columns, they will interfere with our analyses

data.isnull().sum()

ID             0
bg_icd       124
bg_icd_sp    125
ICD          124
dtype: int64

In [26]:
# Count the number of unique ICD-10 codes

data['bg_icd'].nunique()

372

In [27]:
# Get a list of all unique ICD-10 codes in the dataset
# A quick look at this list will inform whether additional cleaning steps need to be added to the steps below

print(data.bg_icd.unique().tolist())

['Q826', 'M95.2', '751610', '753410', 'Q540', 'Q900', nan, '756710', 'Q381', 'Q69.9', 'Q25. 0', 'Q04.0', 'Q25.0', '742100', 'P520', 'Q60.6', 'Q231', '743202', 'Q620', 'D709', 'Q21. 0', 'Q438', 'Q27.0', 'Q82.5', 'Q79.0', 'Q76.3', 'Q82.8', 'Q828', 'D7282', 'Q315', '749201', 'Q250', 'P358', '751490', 'Q21.0', 'Q21.1', 'Q910', 'Q62.0', 'Q17.4', 'Q048', 'Q211', 'L91.8', 'Q549', 'E872', 'Q70.33', 'Q25.9', 'Q02', 'Q98.5', 'Q825', '749090', 'Q55.64', 'Q43.8', 'Q41.1', 'Q22.5', 'D18.1', 'Q210', 'Q38.1', 'P29.3', 'Q99', 'F88', 'Q600', 'D22.5', ' Q65.89', 'Q752', 'Q213', 'Q5569', 'Q21.2', 'Q385', 'Q33.0', 'Q6589', 'Q501', 'Q42.3', 'Q65.89', 'Q659', '750600', '744214', 'Q54.9', '741843', 'Q400', 'Q541', '749190', '758.700', 'O35.8XX0', 'Q18.3', 'Q31.5', 'E2740', 'Q544', 'P362', 'L73.8', 'Q90', 'Q872', 'Q120', 'Q256', 'Q60', 'Q790', 'P70.0', 'P29.12', '745.490', 'Q234', 'Q89.9', 'Q181', '758200', '756190', 'Q04.8', 'N48.82', 'Q04.3', 'Q172', 'Q76.49', 'Q5563', 'Q2549', 'Q5564', 'Q41.9', 'Q892', 'Q6

The cell below conducts basic cleaning on the bg_icd field to match to the Q codes in the reference list. With each new quarterly data import, check to see if any additional cleaning steps are needed, such as additional punctuation or other values that don't belong. \
\
The goal is to match as many ICD codes to the reference list as possible, since the remaining text cleaning and matching processes are more prone to error and misclassification.

In [28]:
# set all codes to uppercase and drop any weird characters from field
data['ICD'] = data['bg_icd'].str.upper() 
remove = ['\\.', ' ', ',', ';', ':', '_', '\\?', '!', '\\(', '\\)']


for i in remove:
    data['ICD'] = data['ICD'].str.replace(i,'',regex=True)

data['ICD'] = data['ICD'].replace("NONEREPORTED",np.NaN)

data[['ID','ICD','bg_icd','bg_icd_sp']]

,ID,ICD,bg_icd,bg_icd_sp
0,1,Q826,Q826,Congenital sacral dimple
1,2,M952,M95.2,"Head, molding noted."
2,3,751610,751610,Mesothelial liver cyst
3,4,753410,753410,Right duplex kidney
4,5,Q540,Q540,"Hypospadias, balanic"
...,...,...,...,...
995,996,Q250,Q250,Patent ductus arteriosus
996,997,756700,756700,Ruptured omphalocele
997,998,Q046,Q046,Congenital cerebral cysts
998,999,Q381,Q38.1,Ankyloglossia


In [29]:
#Restrict the newly cleaned dataset to only those observations that do not have missing ICD or bg_icd_sp

data_new = data.loc[(data['ICD'].notnull()) & (data['ICD'] != "") | (data['bg_icd_sp'].notnull()), :].reset_index().copy()
data_new
data_new.info()

,index,ID,bg_icd,bg_icd_sp,ICD
0,0,1,Q826,Congenital sacral dimple,Q826
1,1,2,M95.2,"Head, molding noted.",M952
2,2,3,751610,Mesothelial liver cyst,751610
3,3,4,753410,Right duplex kidney,753410
4,4,5,Q540,"Hypospadias, balanic",Q540
...,...,...,...,...,...
995,995,996,Q250,Patent ductus arteriosus,Q250
996,996,997,756700,Ruptured omphalocele,756700
997,997,998,Q046,Congenital cerebral cysts,Q046
998,998,999,Q38.1,Ankyloglossia,Q381


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      1000 non-null   int64 
 1   ID         1000 non-null   int64 
 2   bg_icd     876 non-null    object
 3   bg_icd_sp  875 non-null    object
 4   ICD        876 non-null    object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [30]:
data_new.isnull().sum()

index          0
ID             0
bg_icd       124
bg_icd_sp    125
ICD          124
dtype: int64

## Clean bg_icd_sp text

Standardize and clean this text to be able to merge with reference text spreadsheet, and for hardcoding below.

In [33]:
#Clean and turn text into a string
remove = ['\\.', ' ', ',', ';', ':', '_', '\\?', '!', '\\(', '\\)']

data_text = data_new.copy()
for i in remove:
    data_text['bg_icd_sp'] = data_text['bg_icd_sp'].str.replace(i,'', regex=True)
    
text = list(data_text['bg_icd_sp'])

text

['Congenital sacral dimple',
 'Head molding noted',
 'Mesothelial liver cyst',
 'Right duplex kidney',
 'Hypospadias balanic',
 'Trisomy 21 nonmosaicism meiotic nondisjunction',
 'Small apical anterior muscular ventricular septal defect measuring 2mm with bidirectional shunting',
 'gastroschisis',
 'Ankyloglossia',
 'Polydactyly unspecified',
 'Trivial patent ductus arteriosus',
 nan,
 'Patent ductus ateriosus',
 'microcephaly',
 'Small PDA',
 'gastroschisis',
 'Intraventricular hemorrhage grade 1 of newborn',
 'Ankyloglossia',
 'Potter syndrome',
 'Congenital insufficiency of aortic valve',
 'congenital glaucoma right eye',
 'Congenital hydronephrosis',
 'Neutropenia unspecified',
 'Small apical muscular ventricular septal defect VSD',
 'Intraventricular hemorrhage grade 1 of newborn',
 'ATRIAL SEPTAL DEFECT',
 'Other specified congenital malformations of intestine',
 'Congenital absence and hypoplasia of umbilical artery',
 'Mongolian Spots',
 'Congenital Diaphragmatic Hernia',
 'Con

A significant number of observations are coming in as camel case text (for example: HeartDisease). We discovered that camel case interferes with the fuzzy matching process and results in a lower similarity index that might otherwise cause us to throw out a true match. The step below fixes the camel case text by adding a space in between text with a lowercase letter followed immediately by an uppercase letter and then makes all text values lowercase.

In [ ]:
#Fix camel case text
text_new=[]

for i in text:
    i = str(i)
    a = re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1',i).lower()
    text_new.append(a)

Now that the text has been cleaned and standardized, we will merge the cleaned bg_icd_sp field back into the pob3 dataframe so we can use this later when we merge with the ICD text reference spreadsheet and when we perform the NLP steps.

In [ ]:
#Merge the clean text field back into merged_dta for later use with NLP
bg_icd_sp_clean = pd.DataFrame(text_new, columns = ['bg_icd_sp_clean']).reset_index()

data_clean = pd.merge(data_new, bg_icd_sp_clean, left_index=True, right_index=True)
data_clean.drop(['index_x', 'index_y'], axis=1, inplace=True)
data_clean['ICD'] = data_clean['ICD'].astype(str)

#Drop old columns
data_clean=data_clean.drop(['bg_icd','bg_icd_sp'], axis=1) #'level_0'], axis=1) #removing level_0 from drop
data_clean.info()

## Merge POB data to Q code reference list

Now that the ICD codes have been cleaned and we've set those we can based on their reviewed text, we can merge the POB data and the reference list together.

In [ ]:
merged = data_clean.merge(q, how='left', on='ICD')

merged.info()

# View NaN values in Category in the resulting merged dataset
pd.set_option('display.max_rows', None)
merged[merged["Category"].isnull()]

In [ ]:
# Check that the shape of the resulting dataset meets expectations (e.g., number of columns, number of rows)
merged.shape

# Run a quick summary of numbers of observations in each category
merged.groupby('Category').count().sort_values('Text',ascending=False)


## Match MACDP codes

Some jurisdictions use MACDP codes, which are numbers and start with 7 usually. The columns MACDP_low and	MACDP_high contain the reference values we will use to match the MACDP code with the correct birth defect text and category.


In [ ]:
#Restrict to rows where code starts with 7 (presumptive MACDP codes)
MACDP_data = merged.loc[merged['ICD'].str.startswith('7')].copy()

#Convert ICD to numeric
MACDP_data['ICD'] = MACDP_data['ICD'].astype(int)

#Restrict to columns needed
MACDP_data = MACDP_data[['ID','ICD']]
MACDP_data.info()

In [ ]:
#Creating MACDP dictionary
MACDP = q.loc[q['MACDP_low'].notnull(), :].copy()
MACDP.head()
#Restrict to columns needed
MACDP = MACDP[['MACDP_low','MACDP_high','Text','Category']] 

In [ ]:
#Convert ICD 9 columns to integers
MACDP['MACDP_low']=MACDP['MACDP_low'].astype(int)
MACDP['MACDP_high']=MACDP['MACDP_high'].astype(int)

In [ ]:
#Merge dataframes and assign text and category for each row in MACDP data
MACDP_data_assigned = MACDP_data.assign(key=1).merge(MACDP.assign(key=1), on='key')\
                 .drop(columns='key')\
                 .query('ICD.between(MACDP_low, MACDP_high)')\
                 .drop(columns=['MACDP_low', 'MACDP_high'])\
                 .reset_index(drop=True)
MACDP_data_assigned['ICD'] = MACDP_data_assigned['ICD'].astype(str)
MACDP_data_assigned.info()

In [ ]:
#Merge assigned MACDP cases back into full dataframe
merged = merged[['ID','ICD','bg_icd_sp_clean','Text','Category']] 
merged_new= merged.merge(MACDP_data_assigned, how='left', on=['ID','ICD'])

merged_new.head()
merged_new.info()

In [ ]:
#Replace Text_y column with values from Text_x where Text_y is NaN, and rename Text_y
merged_new['Text'] = merged_new['Text_x'].fillna(merged_new['Text_y'])
merged_new['Category'] = merged_new['Category_x'].fillna(merged_new['Category_y'])

merged_new.info()

## Hard-code categorizations for specific observations

Some ICD codes and text have been incorrectly reported in the bg_icd and bg_icd_sp variables and are not, in fact, birth defects. Others provide too little information to categorize, such as test results that are pending. Additionally, some text values occur frequently enough that it's worth hard-coding those observations into their specific categories to ensure the steps above have classified them appropriately and to improve overall accuracy of the NLP model. 

The steps below conduct hard-coding of these categorization in the merged_dta dataframe. With new data submissions, additional hard-coding may be required. Refer to the analyst SOP and clinical review SOP for additional information.

In [ ]:
#Drop old columns
merged_clean=merged_new.drop(['Category_x','Category_y','Text_x','Text_y'], axis=1)
merged_clean.shape
merged_clean.head(20)

Create a separate dataset of all observations with missing ICD code or non-matching ICD codes.

In [ ]:
Qmiss = merged_clean.loc[merged_clean['Category'].isnull(), :].copy()
Qmiss.shape

In [ ]:
Qmiss.info()

Now re-categorize observations with text that contains the words "pending" or "suspected" as unable to categorize, and also re-categorize observations with text or ICD codes that indicate something that is not actually a birth defect. Add some additional categories for hard-coding: 1) observations that are not a birth defect based on ICD code (e.g., non-Q codes), and 2) commonly occurring text (or text values that otherwise need to be hard-coded). For more information, see the analytic SOP and clinical review SOP.

In [ ]:
# Recoding observations as "Not a birth defect of interest/Unable to classify"
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean']=='other'),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean']=='other '),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean']=='a'),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean']=='other anomaly'),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean']=='other  anomaly'),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains("pending")),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains("suspected")),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.find("tongue tie",0)>=0),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('sacral dimple')),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains("possible")),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['ICD'].str.contains("Notabirthdefectofinterest/Unabletoclassify")),'Not a birth defect of interest/Unable to classify',Qmiss['Category'])

# Recoding observations as "Not a birth defect" based on text data
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('murmur')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.find("mongolian",0)>=0),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('nevus')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('tachycardia')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('glycemia')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('non-viable fetus')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('small for gestational age')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('large for gestational age')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('caput')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('hernia umbilical')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('umbilical hernia')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('phimosis')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('hydrocele')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('anemia')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('premature infant')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('stillbirth')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['bg_icd_sp_clean'].str.contains('respiratory distress')),'Not a birth defect',Qmiss['Category'])

# Recoding observations as "Not a birth defect" based on ICD code
Qmiss.loc[Qmiss['ICD'].str.contains(r"^[a-pr-zA-PR-Z][0-9]+$", regex=True), 'Category'] = 'Not a birth defect'

Qmiss['Category'] = np.where((Qmiss['ICD'].str.contains('TERMINALMECOMIUN')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['ICD'].str.contains('FORAMENOVULEANEURYSM')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['ICD'].str.contains('VENTRA')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['ICD'].str.contains('ANEMIA')),'Not a birth defect',Qmiss['Category'])
Qmiss['Category'] = np.where((Qmiss['ICD'].str.contains('Notabirthdefect')),'Not a birth defect',Qmiss['Category'])

Qmiss['Category'].value_counts()

Now we can remove these rows that have been hardcoded from Qmiss and will later add them into the NoMiss dataframe

In [ ]:
Qmiss2 = Qmiss.loc[Qmiss['Category'].isnull(), :].copy()

Qmiss2.shape

nomiss_new = Qmiss.loc[Qmiss['Category'].notnull(), :].copy()

nomiss_new.shape

In [ ]:
#Restrict Qmiss to relevant columns
Qmiss2 = Qmiss2[['ID','bg_icd_sp_clean','ICD']]

The reference list of Q codes that we are using is a living document. It represents all Q codes that are currently present within the POB data. With each new data import, we need to manually check the list of nonmatching ICD codes in Qmiss to see if there are any new Q codes that haven't previously been included in the dataset. These codes should be added to the reference list and categorized accordingly based on clinical review.\
\
IMPORTANT: After the new Q codes have been categorized, you must re-run each of the steps above so that the ICD codes in the POB dataset can be matched with the newly-categorized ICD codes. Remember, the more observations we can match on ICD code, the more robust our model will be, and the fewer observations we will have left to try to predict their categorizations using NLP.

In [ ]:
# Review non-matching Q codes to see if any need to be added to the reference list
Qmiss_review = Qmiss2[Qmiss2['ICD'].astype(str).str.contains('Q')].copy()
Qmiss_review

# If the list is too long to review in the Jupyter notebook, you can export to an excel file using the code below:
Qmiss_review.to_excel('Qmiss_review.xlsx')

Create a separate dataset of all observations with matching ICD codes, which we will use to build our NLP model.

In [ ]:
nomiss = merged_clean.loc[merged_clean['bg_icd_sp_clean'].notnull() & (merged_clean['Category'].notnull()), :].copy()

nomiss.shape

In [ ]:
nomiss2 = pd.concat([nomiss, nomiss_new])

nomiss2.shape

Plot the frequencies of observations for each birth defect category in nomiss. This will give us an idea of the number of data elements we are supplying for the model, and indicates that stratified sampling is necessary for the model because our data is highly imbalanced between categories.

In [ ]:
chart = sns.countplot(x=nomiss2['Category'], order=pd.value_counts(nomiss2['Category']).index)
sns.set(rc = {'figure.figsize':(20,5)},font_scale=2)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

## Fuzzy matching: Non-categorized observations

Observations with missing or nonmatching ICD values in Qmiss might have text in bg_icd_sp_clean that matches exactly or very nearly to a Q code ICD description. If we can match some observations through fuzzy matching, we can add those matched values to nomiss, develop a more robust NLP model, and we are left with fewer unclassified observations that we would have to predict using ML.\
\
We will use the fuzzywuzzy package for Python to conduct the fuzzy matching against the ICD code description text in the reference list.

In [ ]:
#Turn text from q2 into a list of strings and make all characters lowercase
ref_vals = list(q['Text'].str.lower())
ref_vals

In [ ]:
#Turn text from Qmiss into a string
unmatched = list(Qmiss2['bg_icd_sp_clean'])
unmatched

Use the process function in the fuzzywuzzy package to identify the reference ICD code description value with the closest match to the bg_icd_sp text in Qmiss. The step below prints out the ICD code description that is the closest match to the text field in one row and the similarity index for the match in the following row. We create a for loop that will run each bg_icd_sp value through the entire list of ICD code descriptions from the reference list and put the results into a dataframe.\
\
Note that this step can take a few minutes to run on your computer.

In [ ]:
match_results = {'Matched Value':[], 'FW_score':[], 'bg_icd_sp_clean':[]}

highest = process.extractOne(i, ref_vals)

for i in unmatched:
    highest = process.extractOne(i,ref_vals)
    match_results['Matched Value'].append(highest[0])
    match_results['FW_score'].append(highest[1])
    match_results['bg_icd_sp_clean'].append(i)

        
clean_fw_results = pd.DataFrame(match_results)
clean_fw_results

Now that we have the fuzzy matching results lined up nicely to the bg_icd_sp_clean values, we'll merge these results with the original Qmiss dataframe so that these values are linked to participant ID. Note that when you perform the steps below, you may observe that a handful of observations were dropped from the steps above. This is not a mistake! In the current dataset, there are a few IDs that have duplicated bg_icd and bg_icd_sp fields (e.g., the same ICD code and text description repeated twice for the same participant ID in bg_icd1/bg_icd1_sp and bg_icd2/bg_icd2_sp). These are genuine duplicates and should be dropped. If you want to be extra sure, you can export Qmiss to an excel workbook and use the "highlight duplicate values" feature in conditional formatting to do a manual check and ensure it is only true duplicates that are being dropped in the next step.

In [ ]:
clean_fw_results.info()

In [ ]:
#Now combine with original dataframe
miss_fw_matched = pd.merge(Qmiss2, clean_fw_results, how='left', on='bg_icd_sp_clean').drop_duplicates()

miss_fw_matched.rename(columns={'ICD':'ICD_orig'}, inplace = True)
miss_fw_matched.shape

Now we will merge with the reference dataset on ICD description values so that we have the associated category for the matched values. Note that we will need to drop duplicates again, but this time, it's for another reason. The reference list includes some parent ICD codes that have the same ICD text description as certain child codes (e.g., parent and child codes for cleft lip). When we match the two dataframes on ICD description values, we end up with duplicate values for those observations for all but ICD code, because the text description matches to more than one ICD code. To solve this, we drop all values that have a duplicate ID, bg_icd, bg_icd_sp, and Text field. It doesn't matter which ICD code is retained for those values because all we're interested in ultimately is the Category, and the codes with matching ICD text will be assigned to the same Category of birth defects.

In [ ]:
# Merge with reference dataset
qnew = q
qnew['Matched Value'] = q['Text'].str.lower()


miss_fw_matched2 = pd.merge(miss_fw_matched, qnew, how='left', on='Matched Value')

# Drop duplicate observations where ID, bg_icd_sp, and Text are all the same.
miss_fw_matched2 = miss_fw_matched2.drop_duplicates(subset=['ID','bg_icd_sp_clean','Text'])

miss_fw_matched2.shape

Now you can export this dataframe to an excel document that will allow for clinical review of the fuzzy match results. Currently, our cutoff value is a similarity score of 90% or higher for something to be considered a "true" match. For observations with a cutoff value of 90% or higher, we will assign the associated birth defect category to the matching ICD code and text description. However, as new data submissions come in, you will want to check the results as described in the clinical review SOP to ensure that 90% is still an approporiate cutoff value. Use the step below to export the dataframe, updating the date of the filename accordingly.

In [ ]:
# Export an excel file with these results

miss_fw_matched2.to_excel('Fuzzy_Matching_Qmiss.xlsx')

## Preparing the data for ML

Now that we've done fuzzy matching, we want to update the Qmiss and noMiss dataframes to accommodate these additional values before we develop our NLP model. 

miss_fw_matched2 now has a number of observations that are classified due to a fuzzy match similarity score of 90% or greater or due to hard-coding of observations. We want to extract these observations and add them to nomiss, which will increase the robustness of our training and testing data for NLP. To do this, we will flag all of the observations in Qmiss that have now been accurately categorized.

In [ ]:
# Pull out values from miss_fw_matched2 that are now classified due to a fuzzy match score of >=90%
# Create a flag variable
miss_fw_matched2['classified_flag'] = np.where((miss_fw_matched2['FW_score'] > 89),1,0)

miss_fw_matched2.shape

Classified_flag values that equal 1 will be extracted from Qmiss_clean2 and joined to a final version of nomiss to be used to build our NLP model.

In [ ]:
# Remove values to be added to Nomiss
miss_fw_matched3 = miss_fw_matched2.loc[miss_fw_matched2['classified_flag']==1].drop(['Matched Value',
                                                                           'FW_score','classified_flag','ICD_orig'],axis='columns').copy()
miss_fw_matched3.info()

In [ ]:
# Join with nomiss
nomiss3 = pd.concat([nomiss2, miss_fw_matched3],ignore_index = True)
nomiss3.shape
nomiss3=nomiss3.drop(['MACDP_low','MACDP_high'], axis=1)

Now subset miss_fw_matched2 to only those observations with a classified_flag value of 0. These are the remaining uncategorized observations in the dataset that will be categorized using NLP.

In [ ]:
# Subset to remaining uncategorized values for categorization using NLP modeling
Qmiss3 = miss_fw_matched2.loc[miss_fw_matched2['classified_flag']==0].drop(
     ['classified_flag','Matched Value','FW_score','Category','ICD', 'Text','MACDP_low','MACDP_high'],axis='columns').reset_index().copy()
Qmiss3.rename(columns={'ICD_orig':'ICD'}, inplace = True)
Qmiss3.shape

# Now it's time for some NLP!

We will build our NLP classification model using nomiss3, test various models to identify the one with the greatest accuracy, and run the final model on Qmiss3 to categorize the uncategorized text.

We are using the sklearn package for the NLP and machine learning. First we need to split our data into training and validation datasets based on our text data ('bg_icd_sp_clean') and associated categorizations ('Category'). Because our data is heavily imbalanced with each category of birth defects occurring with a different frequency, we need to stratify our training and validation datasets by category. We are using a 75%/25% training/validation split, and we've indicated a random state seed for the purposes of reproducibility (but the steps below could also be run without indicating a random state seed number).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#split into training and testing data
X_train, X_valid, y_train, y_valid = \
    train_test_split(nomiss3['bg_icd_sp_clean'], nomiss3['Category'], stratify=nomiss3['Category'],
                     test_size=0.25, random_state=42)

Now we need to use TfidfVectorizer to transform and process our text data for the model. Our first approach is to do this using unigrams, and we also remove stop words using the built-in function for English stop words such as 'on', 'a', 'the', etc.

There is also an option to determine max features for the vectorizer, but we have removed max features for the time being.

After vectorizing our data, we fit to the training dataset and transform the raw text for each of our training and validation data sets.

In [ ]:
unigrams = TfidfVectorizer(stop_words="english",lowercase=True)

# fit only on the training data
unigrams.fit(X_train)

## transform raw text
tfidf_unigrams_train = unigrams.transform(X_train)
tfidf_unigrams_valid = unigrams.transform(X_valid)

Use get_feature_names to check that the data was vectorized as expected.

In [ ]:
unigrams.get_feature_names_out()

Check the shape of the datasets and the vectorized text.

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_valid.shape)
print(y_valid.shape)

In [ ]:
tfidf_unigrams_train

In [ ]:
tfidf_unigrams_valid

Now we will vectorize the data again but for the purposes of performing cross-validation of each of our models. The difference here is that for cross-validation, the full dataset is vectorized, and not just the training data, as above.

In [ ]:
# Cross-validation
unigrams_cv = TfidfVectorizer(stop_words='english', lowercase= True)

# For cross-validation, fit on all of the training and testing data
unigrams_cv.fit(nomiss3['bg_icd_sp_clean'])

tfidf_unigrams_cv = unigrams_cv.transform(nomiss3['bg_icd_sp_clean'])

In [ ]:
tfidf_unigrams_cv

For cross-validation, we also need to create a separate text array that contains our categorized data.

In [ ]:
y_cv = np.asarray(nomiss3['Category'])
y_cv

Now we will go through each of the steps above again but this time to create vectorized data as bigrams.

In [ ]:
bigrams = TfidfVectorizer(stop_words="english",lowercase=True, ngram_range=(2,2))

# fit only on the training data
bigrams.fit(X_train)

## transform raw text
tfidf_bigrams_train = bigrams.transform(X_train)
tfidf_bigrams_valid = bigrams.transform(X_valid)

In [ ]:
bigrams.get_feature_names_out()

In [ ]:
tfidf_bigrams_train

In [ ]:
tfidf_bigrams_valid

In [ ]:
# Cross-validation
bigrams_cv = TfidfVectorizer(stop_words='english', lowercase= True, ngram_range=(2,2))

# For cross-validation, fit on all of the training and testing data
bigrams_cv.fit(nomiss3['bg_icd_sp_clean'])

tfidf_bigrams_cv = bigrams_cv.transform(nomiss3['bg_icd_sp_clean'])

In [ ]:
tfidf_bigrams_cv

Now we will go through each of the steps above again but this time to create vectorized data as trigrams.

In [ ]:
trigrams = TfidfVectorizer(stop_words="english",lowercase=True, ngram_range=(3,3))

# fit only on the training data
trigrams.fit(X_train)

## transform raw text
tfidf_trigrams_train = trigrams.transform(X_train)
tfidf_trigrams_valid = trigrams.transform(X_valid)

In [ ]:
trigrams.get_feature_names_out()

In [ ]:
tfidf_trigrams_train

In [ ]:
tfidf_trigrams_valid

In [ ]:
# Cross-validation
trigrams_cv = TfidfVectorizer(stop_words='english', lowercase= True, ngram_range=(3,3))

# For cross-validation, fit on all of the training and testing data
trigrams_cv.fit(nomiss3['bg_icd_sp_clean'])

tfidf_trigrams_cv = trigrams_cv.transform(nomiss3['bg_icd_sp_clean'])

In [ ]:
tfidf_trigrams_cv

# Developing NLP model using unigrams

We are testing three different models to see which one has the highest accuracy: Multinomial Naive Bayes, Multi-layer Perceptron (MLP) Classifier, and Random Forest Classifier. We will run each model using unigrams, bigrams, and trigrams, and compare accuracy scores to see which approach classifies our data best.

The first model we will test with unigrams is Multinomial Naive Bayes.

In [ ]:
mod1_nb = MultinomialNB().fit(tfidf_unigrams_train, y_train)

# train results 
mod1_nb_train = mod1_nb.predict(tfidf_unigrams_train)
print('accuracy', accuracy_score(y_train, mod1_nb_train))
print('confusion matrix\n', confusion_matrix(y_train, mod1_nb_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, mod1_nb_train))

# validation results
mod1_nb_valid = mod1_nb.predict(tfidf_unigrams_valid)
print('accuracy', accuracy_score(y_valid, mod1_nb_valid))
print('confusion matrix\n', confusion_matrix(y_valid, mod1_nb_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, mod1_nb_valid))

Now that we have run Multinomial Naive Bayes using unigrams with the training and validation datasets, we will run a cross-validation of this model.

In [ ]:
# Cross-validation of first model
mod1_nb = MultinomialNB()
#scores = cross_val_score(mod1_nb, tfidf_unigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
mod1_nb = MultinomialNB()
scores = cross_val_score(mod1_nb, tfidf_unigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
mod1_nb_cv = cross_val_predict(mod1_nb, tfidf_unigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.accuracy_score(y_cv,mod1_nb_cv)

Now we will follow the same steps above to run our unigrams with our second model, MLP Classifier.

In [ ]:
clf_mlp = MLPClassifier(random_state=1, max_iter=500).fit(tfidf_unigrams_train, y_train)

# train results 
clf_mlp_train = clf_mlp.predict(tfidf_unigrams_train)
print('accuracy', accuracy_score(y_train, clf_mlp_train))
print('confusion matrix\n', confusion_matrix(y_train, clf_mlp_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, clf_mlp_train))

# validation results
clf_mlp_valid = clf_mlp.predict(tfidf_unigrams_valid)
print('accuracy', accuracy_score(y_valid, clf_mlp_valid))
print('confusion matrix\n', confusion_matrix(y_valid, clf_mlp_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, clf_mlp_valid))

In [ ]:
# Cross-validation of second model
#clf_mlp = MLPClassifier(random_state=1, max_iter=500)
#scores = cross_val_score(clf_mlp, tfidf_unigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
clf_mlp = MLPClassifier(random_state=1, max_iter=500)
scores = cross_val_score(clf_mlp, tfidf_unigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
clf_mlp_cv = cross_val_predict(clf_mlp, tfidf_unigrams_cv, y_cv, cv=5, n_jobs=-1)
#sklearn.metrics.accuracy_score(y_cv, clf_mlp_cv)

The training and validation datasets performed much better using the MLP Classifier model, although the cross-validation scores were considerably lower. 

Now we will try the random forest classifier model using unigrams.

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1).fit(tfidf_unigrams_train, y_train)

# train results 
clf_rf_train = clf_rf.predict(tfidf_unigrams_train)
print('accuracy', accuracy_score(y_train, clf_rf_train))
print('confusion matrix\n', confusion_matrix(y_train, clf_rf_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, clf_rf_train))

# validation results
clf_rf_valid = clf_rf.predict(tfidf_unigrams_valid)
print('accuracy', accuracy_score(y_valid, clf_rf_valid))
print('confusion matrix\n', confusion_matrix(y_valid, clf_rf_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, clf_rf_valid))

In [ ]:
# Cross-validation of third model
#clf_rf = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1)
#scores = cross_val_score(clf_rf, tfidf_unigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
clf_rf = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1)
scores = cross_val_score(clf_rf, tfidf_unigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
clf_rf_cv = cross_val_predict(clf_rf, tfidf_unigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.accuracy_score(y_cv,clf_rf_cv)

Among unigrams, the model with the highest accuracy scores is the MLP Classifier model, built on the training dataset. We will use that model to categorize our unclassified data.

# Developing NLP model using bigrams

We want to continue our model testing and development by exploring how the accuracy of our models change when we use bigrams instead of unigrams. We will conduct all the same steps above for bigrams and compare the accuracy scores to the unigrams.

In [ ]:
#First model: Multinomial Naive Bayes
mod2_nb = MultinomialNB().fit(tfidf_bigrams_train, y_train)

# train results 
mod2_nb_train = mod2_nb.predict(tfidf_bigrams_train)
print('accuracy', accuracy_score(y_train, mod2_nb_train))
print('confusion matrix\n', confusion_matrix(y_train, mod2_nb_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, mod2_nb_train))

# validation results
mod2_nb_valid = mod2_nb.predict(tfidf_bigrams_valid)
print('accuracy', accuracy_score(y_valid, mod2_nb_valid))
print('confusion matrix\n', confusion_matrix(y_valid, mod2_nb_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, mod2_nb_valid))

In [ ]:
# Cross-validation of first model
#mod2_nb = MultinomialNB()
#scores = cross_val_score(mod2_nb, tfidf_bigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
mod2_nb = MultinomialNB()
scores = cross_val_score(mod2_nb, tfidf_bigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
mod2_nb_cv = cross_val_predict(mod2_nb, tfidf_bigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.accuracy_score(y_cv,mod2_nb_cv)

In [ ]:
# Second model: MLP Classifier
clf_mlp2 = MLPClassifier(random_state=1, max_iter=500).fit(tfidf_bigrams_train, y_train)

# train results 
clf_mlp2_train = clf_mlp2.predict(tfidf_bigrams_train)
print('accuracy', accuracy_score(y_train, clf_mlp2_train))
print('confusion matrix\n', confusion_matrix(y_train, clf_mlp2_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, clf_mlp2_train))

# validation results
clf_mlp2_valid = clf_mlp2.predict(tfidf_bigrams_valid)
print('accuracy', accuracy_score(y_valid, clf_mlp2_valid))
print('confusion matrix\n', confusion_matrix(y_valid, clf_mlp2_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, clf_mlp2_valid))

In [ ]:
# Cross-validation of second model
#clf_mlp2 = MLPClassifier(random_state=1, max_iter=500)
#scores = cross_val_score(clf_mlp2, tfidf_bigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
clf_mlp2 = MLPClassifier(random_state=1, max_iter=500)
scores = cross_val_score(clf_mlp2, tfidf_bigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
clf_mlp2_cv = cross_val_predict(clf_mlp2, tfidf_bigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.accuracy_score(y_cv,clf_mlp2_cv)

In [ ]:
# Third model: Random Forest Classifier
clf_rf2 = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1).fit(tfidf_bigrams_train, y_train)

# train results 
clf_rf2_train = clf_rf2.predict(tfidf_bigrams_train)
print('accuracy', accuracy_score(y_train, clf_rf2_train))
print('confusion matrix\n', confusion_matrix(y_train, clf_rf2_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, clf_rf2_train))

# validation results
clf_rf2_valid = clf_rf2.predict(tfidf_bigrams_valid)
print('accuracy', accuracy_score(y_valid, clf_rf2_valid))
print('confusion matrix\n', confusion_matrix(y_valid, clf_rf2_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, clf_rf2_valid))

In [ ]:
# Cross-validation of third model
#clf_rf2 = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1)
#scores = cross_val_score(clf_rf2, tfidf_bigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
clf_rf2 = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1)
scores = cross_val_score(clf_rf2, tfidf_bigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
clf_rf2_cv = cross_val_predict(clf_rf2, tfidf_bigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.accuracy_score(y_cv, clf_rf2_cv)

# Developing NLP model using trigrams

Finally, we will run through all of these steps again using trigrams to see if model performance is improved compared to unigrams and bigrams.

In [ ]:
#First model: Multinomial Naive Bayes
mod3_nb = MultinomialNB().fit(tfidf_trigrams_train, y_train)

# train results 
mod3_nb_train = mod3_nb.predict(tfidf_trigrams_train)
print('accuracy', accuracy_score(y_train, mod3_nb_train))
print('confusion matrix\n', confusion_matrix(y_train, mod3_nb_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, mod3_nb_train))

# validation results
mod3_nb_valid = mod3_nb.predict(tfidf_trigrams_valid)
print('accuracy', accuracy_score(y_valid, mod3_nb_valid))
print('confusion matrix\n', confusion_matrix(y_valid, mod3_nb_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, mod3_nb_valid))

In [ ]:
# Cross-validation of first model
#mod3_nb = MultinomialNB()
#scores = cross_val_score(mod3_nb, tfidf_trigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
mod3_nb = MultinomialNB()
scores = cross_val_score(mod3_nb, tfidf_trigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
mod3_nb_cv = cross_val_predict(mod3_nb, tfidf_trigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.accuracy_score(y_cv, mod3_nb_cv)

In [ ]:
# Second model: MLP Classifier
clf_mlp3 = MLPClassifier(random_state=1, max_iter=500).fit(tfidf_trigrams_train, y_train)

# train results 
clf_mlp3_train = clf_mlp3.predict(tfidf_trigrams_train)
print('accuracy', accuracy_score(y_train, clf_mlp3_train))
print('confusion matrix\n', confusion_matrix(y_train, clf_mlp3_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, clf_mlp3_train))

# validation results
clf_mlp3_valid = clf_mlp3.predict(tfidf_trigrams_valid)
print('accuracy', accuracy_score(y_valid, clf_mlp3_valid))
print('confusion matrix\n', confusion_matrix(y_valid, clf_mlp3_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, clf_mlp3_valid))

In [ ]:
# Cross-validation of second model
#clf_mlp3 = MLPClassifier(random_state=1, max_iter=500)
#scores = cross_val_score(clf_mlp3, tfidf_trigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
clf_mlp3 = MLPClassifier(random_state=1, max_iter=500)
scores = cross_val_score(clf_mlp3, tfidf_trigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
clf_mlp3_cv = cross_val_predict(clf_mlp3, tfidf_trigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.f1_score(y_cv, clf_mlp3_cv, average='weighted')

In [ ]:
# Third model: Random Forest Classifier
clf_rf3 = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1).fit(tfidf_trigrams_train, y_train)

# train results 
clf_rf3_train = clf_rf3.predict(tfidf_trigrams_train)
print('accuracy', accuracy_score(y_train, clf_rf3_train))
print('confusion matrix\n', confusion_matrix(y_train, clf_rf3_train))
print('(row=expected, col=predicted)')
print(classification_report(y_train, clf_rf3_train))

# test results
clf_rf3_valid = clf_rf3.predict(tfidf_trigrams_valid)
print('accuracy', accuracy_score(y_valid, clf_rf3_valid))
print('confusion matrix\n', confusion_matrix(y_valid, clf_rf3_valid))
print('(row=expected, col=predicted)')
print(classification_report(y_valid, clf_rf3_valid))

In [ ]:
# Cross-validation of third model
#clf_rf3 = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1)
#scores = cross_val_score(clf_rf3, tfidf_trigrams_cv, y_cv, scoring='accuracy', cv=5, n_jobs=-1)
#print(scores)
#print(np.mean(scores))
clf_rf3 = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1)
scores = cross_val_score(clf_rf3, tfidf_trigrams_cv, y_cv, scoring='f1_weighted', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

In [ ]:
clf_rf3_cv = cross_val_predict(clf_rf3, tfidf_trigrams_cv, y_cv, cv=5, n_jobs=-1)
sklearn.metrics.f1_score(y_cv, clf_rf3_cv, average='weighted')

## Merge the classified results into a single dataset

Since the unigrams results with MLP Classifier were the most accurate, we will consider these to be our true categories and will merge their data with nomiss_clean3 so we have a full, classified dataset.

In [ ]:
Qmiss4 = Qmiss3[['ID','ICD','bg_icd_sp_clean']].copy()
Qmiss4.shape

In [ ]:
# Model with the highest accuracy
clf_mlp = MLPClassifier(random_state=1, max_iter=500).fit(tfidf_unigrams_train, y_train)

# transform raw text in Qmiss
tfidfQmiss_uni = unigrams.transform(Qmiss3['bg_icd_sp_clean'])
tfidfQmiss_uni

In [ ]:
classified_data = pd.Series(clf_mlp.predict(tfidfQmiss_uni))
classified_data

In [ ]:
Qmiss_classified = pd.concat([Qmiss4,classified_data],axis=1)
Qmiss_classified.rename(columns={0:"Category"},inplace=True)

In [ ]:
#Save separate dataset for just the Qmiss observations categorized 
# by our model
Qmiss_classified.to_excel('Missing Q codes_classified.xlsx')

In [ ]:
# Join Qmiss_classified with nonmissing
POB_final = pd.concat([nomiss3, Qmiss_classified],ignore_index = True)
POB_final

In [ ]:
# Export the final dataset
# Be sure to change the date in the filename!
POB_final.to_excel('POB Birth Defects Classified_NLP.xlsx')

Congratulations! You have now cleaned the birth defects data and categorized each observation into a birth defect category.

If you have any questions about this code, please contact the SET-NET team.